# Tagging the USA by New York Times

### To understand what are the top 10 frequently used tags in all articles every month on the New York Times, and look at overall activities around these tags on other media platforms
- Text Analytics with Sentimental Analysis
- Interpreting the media world - Google searches, Reddit, and Youtube from tags of the Times Perspective


In [1]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [2]:
sys.path

['/Users/hh/Documents/Pratt/Adv.ProjectsinVis/Monthly-Frequency-of-NYTimes-Tag',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Users/hh/Library/Python/3.7/lib/python/site-packages',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/Users/hh/.ipython']

In [3]:
import json
import requests
import pandas as pd
import numpy as np
import datetime
import operator
import time
import praw
import nltk
import configparser

from pytrends.request import TrendReq
from praw.models import MoreComments
from googleapiclient import discovery
from textblob import TextBlob
from pandas.io.json import json_normalize

import pprint
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# Use CofigParser to safely store the password or key
config = configparser.ConfigParser()
config.read('key_pair.ini')

Times_key = config['Times']['key']

Reddit_client_id = config['Reddit']['client_id']
Reddit_client_secret = config['Reddit']['client_secret']
Reddit_username = config['Reddit']['username']
Reddit_password = config['Reddit']['password']
Reddit_user_agent = config['Reddit']['user_agent']

Youtube_YOUTUBE_API_SERVICE_NAME = config['Youtube']['YOUTUBE_API_SERVICE_NAME']
Youtube_YOUTUBE_API_VERSION = config['Youtube']['YOUTUBE_API_VERSION']
Youtube_DEVELOPER_KEY = config['Youtube']['DEVELOPER_KEY']

In [6]:
Times_key

'GeYpgeI2hytY16DuEetpDFOc7l0997Qs'

### New York Times API
https://developer.nytimes.com/apis

Archive API
- Finds all articles's metadata from archive APi (about 7000 every month): title, section_name, category, url, word_count, keywords

In [9]:
def get_init_info(yy, mm):
    if len(mm) == 2 and '0' in str(mm):
        mm = mm[-1]
    parameters = {'api-key': Times_key}           
    archived_Url = 'https://api.nytimes.com/svc/archive/v1/'+ str(yy) +'/'+ str(mm) +'.json'
    archives = requests.get(archived_Url, params=parameters).json()

    each_metadata = []
    for a in archives['response']['docs']:
        articles = {}
        articles['title'] = a['headline']['main']
        articles['pub_date'] = a['pub_date'][:10]
        articles['url'] = a['web_url']
        articles['description'] = a['lead_paragraph']
        if len(a['multimedia']) !=0 and a['multimedia'][0]['url']:
            articles['thm_img'] = 'https://static01.nyt.com/' + a['multimedia'][0]['url']
        else:
            articles['thm_img'] = 'no_image_found'
        articles['tags'] = [''.join(tag['value']) for tag in a['keywords']]
        each_metadata.append(articles)
        
    tag_arr = []
    for m in each_metadata:
        for t in m['tags']:
            for string in t.split(', '):
                tag_arr.append(', '.join(string.split(', ')))
                    
    count_tag = {}
    for tag in tag_arr:
        if tag in ['Trump', 'Donald J']:
            tag = 'Donald Trump'
        if tag in ['Joseph R Jr', 'Biden']:
            tag = 'Joe Biden'
        if tag in ['Brett M', 'Supreme Court (US)', 'Kavanaugh']:
            tag = 'Brett Kavanaugh'
        if tag in ['Putin', 'Vladimir V']:
            tag = 'Putin'
        if tag in ['New York City', 'NYC','NY)', 'NY']:
            tag = 'New York City'
        if tag in ['Fla', 'Parkland']:
            tag = 'Parkland'                          
        if tag in count_tag:
            count_tag[tag] += 1
        else:
            count_tag[tag] = 1   
            
    tags_with_frequency = sorted(count_tag.items(),key=operator.itemgetter(1),reverse=True)[:20]      

    data = []
    for top_tag in tags_with_frequency:
        print(top_tag[0])
        multi_articles = {}
        for each in reversed(each_metadata) :
            if top_tag[0] in multi_articles.values():
                pass      
            else:
                if len(each['tags']) != 0 and top_tag[0] in each['tags']:
                    multi_articles['tag'] = top_tag[0]    
                    multi_articles['frequency'] = str(top_tag[1])
                    multi_articles['title'] = each['title']
                    multi_articles['pub_date'] = each['pub_date']
                    multi_articles['url'] = each['url']
                    multi_articles['description'] = each['description']
                    multi_articles['thm_img'] = each['thm_img']
                    data.append(multi_articles) 
                    
#     print('tags_with_frequency', tags_with_frequency)       
    return data

In [10]:
get_init_info(today[:4], today[5:7])

Coronavirus (2019-nCoV)
Donald Trump
United States Politics and Government
Presidential Election of 2020
Quarantines
New York City
Joe Biden
Epidemics
Primaries and Caucuses
Democratic Party
United States
Politics and Government
United States Economy
Sanders
Bernard
Shutdowns (Institutional)
Elections
Books and Literature
California
Television


[{'tag': 'Coronavirus (2019-nCoV)',
  'frequency': '3088',
  'title': 'Stock Markets in Asia Signal Lingering Unease: Live Updates',
  'pub_date': '2020-03-30',
  'url': 'https://www.nytimes.com/2020/03/30/business/coronavirus-stock-markets-economy.html',
  'description': 'Markets in the Asia-Pacific region opened broadly lower on Monday, as investors remained nervous about global economic prospects despite stabilization efforts from governments around the world.',
  'thm_img': 'https://static01.nyt.com/images/2020/03/29/business/30markets-briefing1/29VIRUS-VENTILATORS-promo-articleLarge.jpg'},
 {'tag': 'United States Politics and Government',
  'frequency': '870',
  'title': 'White House Airlifts Medical Supplies From China in Coronavirus Fight',
  'pub_date': '2020-03-29',
  'url': 'https://www.nytimes.com/2020/03/29/business/economy/coronavirus-china-supplies.html',
  'description': 'WASHINGTON — A commercial aircraft carrying 80 tons of gloves, masks, gowns and other medical suppli

In [11]:
# Archive API
def get_TimesMetadata_from_TimesArchivedArticles(): #these arguments need to be number
    articles_metadata = {}
    today = str(datetime.datetime.now())
    for yy in range(2018, int(today[:4]) + 1):
        if str(yy) == today[:4]:
            if '0' in today[5:7]:
                ends = int(today[6:7]) + 1
            else:
                ends = int(today[5:7]) + 1
        else:
            ends = 13
        for mm in range(1,ends):
            print(yy,mm)
            parameters = {'api-key': Times_key}           
            archived_Url = 'https://api.nytimes.com/svc/archive/v1/'+ str(yy) +'/'+ str(mm) +'.json'
            archives = requests.get(archived_Url, params=parameters).json()
#             pprint.pprint(archives['response']['docs'])
            articles_all = []
            for a in archives['response']['docs']:
                articles = {}
                articles['title'] = a['headline']['main']
                articles['pub_date'] = a['pub_date'][:10]
                articles['url'] = a['web_url']
                articles['word_count'] = a['word_count']
                articles['tags'] = [''.join(tag['value']) for tag in a['keywords']]# if a['keywords'].index(tag) == 0 or a['keywords'].index(tag) == 1 or a['keywords'].index(tag) == 2]
                articles_all.append(articles)
            articles_metadata[str(yy)+'-'+str(mm)] = articles_all
    return articles_metadata

In [12]:
articles_metadata = get_TimesMetadata_from_TimesArchivedArticles()
articles_metadata['2020-3']
# {'yy-mm': [{'title': 'one_sentence', 'pub_date': 'yy-mm-dd'..}]}

2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
2020 1
2020 2
2020 3


[{'title': 'Living In... Ojai, Calif.',
  'pub_date': '2020-03-03',
  'url': 'https://www.nytimes.com/slideshow/2020/03/03/realestate/living-in-ojai-calif.html',
  'word_count': 0,
  'tags': ['Ojai (Calif)',
   'Los Angeles (Calif)',
   'Real Estate and Housing (Residential)',
   'California']},
 {'title': 'As Coronavirus Hits U.K., Health Workers Fear Getting Sick, and Going Broke',
  'pub_date': '2020-03-03',
  'url': 'https://www.nytimes.com/2020/03/03/world/europe/coronavirus-health-workers-uk.html',
  'word_count': 1144,
  'tags': ['Coronavirus (2019-nCoV)',
   'Labor and Jobs',
   'National Health Service',
   'Workplace Hazards and Violations',
   'Hospitals',
   'Freelancing, Self-Employment and Independent Contracting',
   'Great Britain']},
 {'title': 'In Coronavirus Fight, China Gives Citizens a Color Code, With Red Flags',
  'pub_date': '2020-03-02',
  'url': 'https://www.nytimes.com/2020/03/01/business/china-coronavirus-surveillance.html',
  'word_count': 1498,
  'tags': [

In [13]:
def get_most_popular_viewedArticle():
    parameters = {'api-key': Times_key}           
    archived_Url = 'https://api.nytimes.com/svc/mostpopular/v2/viewed/7.json'
    archives = requests.get(archived_Url, params=parameters).json()
#     pprint.pprint(archives['results'])
    popular_article = []
    for a in archives['results']:
        articles = {}
        articles['title'] = a['title']
        articles['pub_date'] = a['published_date']
        articles['url'] = a['url']
        if a['des_facet']:
            articles['tags'] = a['des_facet']
        if a['media']:
            articles['thm_img'] = a['media'][0]['media-metadata'][2]['url']
        popular_article.append(articles)
    return popular_article

In [14]:
popular_article = get_most_popular_viewedArticle()
popular_article

[{'title': 'F.A.Q. on Stimulus Checks, Unemployment and the Coronavirus Plan',
  'pub_date': '2020-03-26',
  'url': 'https://www.nytimes.com/article/coronavirus-stimulus-package-questions-answers.html',
  'tags': ['Coronavirus (2019-nCoV)',
   'Coronavirus Aid, Relief, and Economic Security Act (2020)',
   'Stimulus (Economic)',
   'Federal Aid (US)',
   'Unemployment Insurance',
   'United States Economy',
   'United States Politics and Government'],
  'thm_img': 'https://static01.nyt.com/images/2020/03/27/business/26virus-qa/26virus-qa-mediumThreeByTwo440.jpg'},
 {'title': 'The U.S. Now Leads the World in Confirmed Coronavirus Cases',
  'pub_date': '2020-03-26',
  'url': 'https://www.nytimes.com/2020/03/26/health/usa-coronavirus-cases.html',
  'tags': ['Politics and Government',
   'United States International Relations',
   'Medicine and Health',
   'Quarantines',
   'Coronavirus (2019-nCoV)',
   'Epidemics',
   'Impeachment'],
  'thm_img': 'https://static01.nyt.com/images/2020/03/2

In [15]:
# Get frequency data
def get_frequentlyUsedTags_from_Times(articles_metadata):
    tags_with_frequency = {}
    today = str(datetime.datetime.now())
    for yy in range(2018, int(today[:4]) + 1):
        if str(yy) == today[:4]:
            if '0' in today[5:7]:
                ends = int(today[6:7]) + 1
            else:
                ends = int(today[5:7]) + 1
        else:
            ends = 13
        for mm in range(1, ends):
            for a in articles_metadata:
                if a == str(yy)+'-'+str(mm):
                    tag_arr = []
                    for each in articles_metadata[a]:
                        for t in each['tags']:
                            for string in t.split(', '):
                                 tag_arr.append(', '.join(string.split(', ')))
                    count_tag = {}
                    tags_only = []
                    for tag in tag_arr:
                        if tag in ['Trump', 'Donald J']:
                            tag = 'Donald Trump'
                        if tag in ['Joseph R Jr', 'Biden']:
                            tag = 'Joe Biden'
                        if tag in ['Brett M', 'Supreme Court (US)', 'Kavanaugh']:
                            tag = 'Brett Kavanaugh'
                        if tag in ['Putin', 'Vladimir V']:
                            tag = 'Putin'
                        if tag in ['New York City', 'NYC','NY)', 'NY']:
                            tag = 'New York City'
                        if tag in ['Fla', 'Parkland']:
                            tag = 'Parkland'                          
                            
                        if tag in count_tag:
                            count_tag[tag] += 1
                        else:
                            count_tag[tag] = 1
                            
                    tags_with_frequency[str(yy)+'-'+str(mm)] = sorted(count_tag.items(),key=operator.itemgetter(1),reverse=True)[:10]

                    for monthly in list(tags_with_frequency.values()):
                        for countWithtag in monthly:
                            tags_only.append(countWithtag[0].replace(' (2018)', '').replace(' (2019)',''))
                    unique_tags_only = list(set(tags_only))         
    return tags_with_frequency, unique_tags_only

In [16]:
tags_with_frequency, unique_tags_only = get_frequentlyUsedTags_from_Times(articles_metadata)
tags_with_frequency['2020-3']
# {'yy-mm': [('tags1', tags1_count),('tags2', tags2_count)], 'yy-mm': [('tags1', tags1_count)]}
# unique_tags_only
# [tags1', tags2', tags3'...]

[('Coronavirus (2019-nCoV)', 3088),
 ('Donald Trump', 1502),
 ('United States Politics and Government', 870),
 ('Presidential Election of 2020', 837),
 ('Quarantines', 813),
 ('New York City', 744),
 ('Joe Biden', 742),
 ('Epidemics', 606),
 ('Primaries and Caucuses', 567),
 ('Democratic Party', 395)]

In [17]:
# Get a unique tag collection for the search query
def get_trends_from_top10_TimesTags(unique_tags_only):
    pytrends = TrendReq(hl='en-US', tz=360)
    tags_dfs = []
    print('Total unique tags: ', len(unique_tags_only))
    for tag in unique_tags_only:
        tag_arr = []
        time.sleep(1)
        tag_arr.append(tag)
        print(tag)
        pytrends.build_payload(tag_arr, cat=0, timeframe='2018-01-01 2020-03-07', geo='', gprop='')
        time.sleep(1)
        interest_df = pytrends.interest_over_time().reset_index()
        
#         sns.lineplot( x = interest_df['date'] , y = interest_df[tag], alpha = 0.8)
#         plt.legend(loc='upper left', bbox_to_anchor=(1,1))
#         plt.xticks( rotation = 90 )
        
    tags_dfs.append(interest_df)
    return tags_dfs

In [18]:
interest_over_time = get_trends_from_top10_TimesTags(unique_tags_only)
interest_over_time[0]

Total unique tags:  44
Illegal Immigration
United States International Relations
Trump-Ukraine Whistle-blower Complaint and Impeachment Inquiry
Joe Biden
Impeachment
Donald Trump
Quarantines
International Trade and World Market
Russia
Senate
Christine Blasey
Weddings and Engagements
House of Representatives
Television
Olympic Games
Women and Girls
United States Politics and Government
Coronavirus (2019-nCoV)
Politics and Government
Russian Interference in 2016 US Elections and Ties to Trump Associates
Presidential Election of 2020
Parkland
Shutdowns (Institutional)
New York City
Shooting
Democratic Party
Books and Literature
Ford
School Shootings and Armed Attacks
Movies
Primaries and Caucuses
#MeToo Movement
Immigration and Emigration
Epidemics
Sanders
Elections
Art
Blacks
Bernard
Putin
Midterm Elections
Sexual Harassment
Brett Kavanaugh
China


,date,China,isPartial
0,2018-01-07,41,False
1,2018-01-14,41,False
2,2018-01-21,41,False
3,2018-01-28,40,False
4,2018-02-04,41,False
...,...,...,...
108,2020-02-02,78,False
109,2020-02-09,64,False
110,2020-02-16,54,False
111,2020-02-23,63,False


### Reddit API
https://praw.readthedocs.io/en/latest/

In [19]:
# Reddit API
def get_reddit_comments(tags):
    reddit = praw.Reddit(client_id = Reddit_client_id,
                         client_secret = Reddit_client_secret,
                         username = Reddit_username,
                         password = Reddit_password,
                         user_agent = Reddit_user_agent)
    reddit_metadata = []
    for tag in tags:
#         if tag in ['OlympicGames', 'Shooting', 'NewYorkCity', 'InternationalTradeandWorldMarket', 'Impeachment', 'MidtermElections', 'Elections', 'Senate', 'Parkland', 'GovernmentShutdowns', 'BrettKavanaugh', 'UnitedStatesPoliticsandGovernment', 'PresidentialElectionof2020', 'PoliticsandGovernment', 'ChristineBlasey', 'WomenandGirls', 'Sanders', 'Immigration', 'Iran']:
#             pass
#         else:
        print(tag)
        subreddit = reddit.subreddit('all')
        each_tag = {}
        for post in subreddit.search(tag, limit=5):
            each_tag['Tag'] = tag
            each_tag['Title'] = post.title
            each_tag['URL'] = post.url
            each_tag_comments = post.comments.list()
            comments_arr = []
            for comment in each_tag_comments:
                if isinstance(comment, MoreComments):
                    continue
                comments_arr.append(comment.body)
            each_tag['Comments'] = comments_arr
        reddit_metadata.append(each_tag)
    return reddit_metadata

In [20]:
reddit_metadata = get_reddit_comments(unique_tags_only)
reddit_metadata[0]
# reddit_metadata = [{'Tag': 'tag1', 'Title': 'title1', 'url': 'url1', 'Comments': ['comment1', 'comment2',...]}]

Illegal Immigration
United States International Relations
Trump-Ukraine Whistle-blower Complaint and Impeachment Inquiry
Joe Biden
Impeachment
Donald Trump
Quarantines
International Trade and World Market
Russia
Senate
Christine Blasey
Weddings and Engagements
House of Representatives
Television
Olympic Games
Women and Girls
United States Politics and Government
Coronavirus (2019-nCoV)
Politics and Government
Russian Interference in 2016 US Elections and Ties to Trump Associates
Presidential Election of 2020
Parkland
Shutdowns (Institutional)
New York City
Shooting
Democratic Party
Books and Literature
Ford
School Shootings and Armed Attacks
Movies
Primaries and Caucuses
#MeToo Movement
Immigration and Emigration
Epidemics
Sanders
Elections
Art
Blacks
Bernard
Putin
Midterm Elections
Sexual Harassment
Brett Kavanaugh
China


{'Tag': 'Illegal Immigration',
 'Title': 'Liberal here, just here to say I agree with y’all about illegal immigration. We don’t owe illegal immigrants shit.',
 'URL': 'https://www.reddit.com/r/Conservative/comments/c9mxsn/liberal_here_just_here_to_say_i_agree_with_yall/',
 'Comments': ['#Butthurt Report\n\n8: This is spam\n\n3: It\'s targeted harassment at someone else\n\n2: Concern Troll\n\n2: Shitpost\n\n1: A liberal named "OK__LIBTARD"? yeah sure\n\n1: Rule 4: No vote brigading. All links to other subs / comments must use the np prefix.\n\n1: lol\n\n1: Rule 5: Shit posts removed at moderator discretion.\n\n1: You\'re a conservative, you fucking idiot. \n\n1: <no reason>\n\n1: It threatens violence or physical harm at someone else\n\n1: It\'s rude, vulgar or offensive\n\n1: obvious troll',
  "I've got a friend that keeps telling me he's liberal but everything he says goes against what the dems have said in the last couple years. Make sure your party didnt leave the station without yo

In [21]:

def get_sentimental_chart(reddit_metadata):
    tags_metadata = []
    for single_tag in reddit_metadata:
        tag_sentiment = {}
        avg_polarity = 0
        pos_count = 0
        neg_count = 0
        neutral_count = 0
        for single_comment in single_tag['Comments']: 
            
            # Get Word Sentimental analysis
            blobed_word = TextBlob(single_comment)
            if blobed_word.sentiment.polarity > 0:
                pos_count += 1
            elif blobed_word.sentiment.polarity == 0:
                neutral_count += 1
            else:
                neg_count += 1      
            avg_polarity += blobed_word.sentiment.polarity
            
            tag_sentiment['tag'] = single_tag['Tag']    
            tag_sentiment['avg_polarity'] = avg_polarity / len(single_tag['Comments'])
            tag_sentiment['pos_count'] = pos_count
            tag_sentiment['neg_count'] = neg_count
            tag_sentiment['neutral_count'] = neutral_count
            
            # Get Adjective words' frequencies
            adgs_arr = []
            for blobed_tags in blobed_word.tags:
                if blobed_tags[1] in ['JJ', 'JJR', 'JJS']:
                    adgs_arr.append(blobed_tags[0])
            adg_frequency = {}
            for adg in adgs_arr: 
                if adg in adg_frequency:
                    adg_frequency[adg] += 1
                else:
                    adg_frequency[adg] = 1 
            tag_sentiment['adg_frequency'] = sorted(adg_frequency.items(),key=operator.itemgetter(1),reverse=True)[:3]       
        tags_metadata.append(tag_sentiment)
        
    return tags_metadata

In [22]:
tags_metadata = get_sentimental_chart(reddit_metadata)
tags_metadata

[{'tag': 'Illegal Immigration',
  'avg_polarity': 0.07282182615017248,
  'pos_count': 210,
  'neg_count': 113,
  'neutral_count': 90,
  'adg_frequency': [('less', 1), ('asholeish', 1), ('Most', 1)]},
 {'tag': 'United States International Relations',
  'avg_polarity': 0.014247921390778528,
  'pos_count': 2,
  'neg_count': 1,
  'neutral_count': 4,
  'adg_frequency': []},
 {'tag': 'Trump-Ukraine Whistle-blower Complaint and Impeachment Inquiry',
  'avg_polarity': 0.02886766233250256,
  'pos_count': 203,
  'neg_count': 151,
  'neutral_count': 137,
  'adg_frequency': [('Most', 1), ('strongman', 1)]},
 {'tag': 'Joe Biden',
  'avg_polarity': 0.031042977696296475,
  'pos_count': 186,
  'neg_count': 132,
  'neutral_count': 139,
  'adg_frequency': [('big', 4), ('less', 2), ('single', 2)]},
 {'tag': 'Impeachment',
  'avg_polarity': 0.008305975742114441,
  'pos_count': 175,
  'neg_count': 178,
  'neutral_count': 135,
  'adg_frequency': [('many', 1)]},
 {'tag': 'Donald Trump',
  'avg_polarity': -0.

### Youtube API
https://developers.google.com/youtube/v3/docs/search/list?hl=en_US

In [23]:
# creating Youtube Resource Object 
youtube_object = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey = DEVELOPER_KEY) 
   
def youtube_search_keyword(unique_tags_only):  
    # calling the search.list method to retrieve youtube search results 
    max_results = 5
    results = []
    for tag in unique_tags_only:
        print(tag)
        search_tags = youtube_object.search().list(q = tag, part = "id, snippet", order = 'viewCount', maxResults = max_results, publishedAfter = "2020-02-17T00:00:00Z").execute() 
        videos = []
        for item in search_tags.get("items", []):
            video = {}
            video["Tag"] = tag
            video["videoId"] = item["id"]["videoId"]
            video["publishedAt"] = item['snippet']['publishedAt']
            video["title"] = item['snippet']['title']
            video['description'] = item['snippet']['description']
            stats = youtube_object.videos().list(part='statistics, snippet', id=item["id"]["videoId"]).execute()
            video['statistics'] = stats.get("items", [])[0]['statistics']
            videos.append(video)
        results.append(videos)
    return results

NameError: name 'build' is not defined

In [ ]:
youtube_search_keyword(unique_tags_only)

#### [Not using anymore] 

- Top Stories API: (about 30 articles by 7 days): title, pub_date, url, section, des_facet, geo_facet
- Newswire API: Finds by Archive API's url : des_facet
- Most Popular API
- Top Stories API, Times Tags API, Community API